In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 42.9 MB/s eta 0:00:00


In [2]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

model = AutoModelForCausalLM.from_pretrained(
                                              pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
                                              quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=getattr(torch, "float16"), bnb_4bit_quant_type="nf4")
                                            )
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

training_arguments = TrainingArguments(
    output_dir = "./results",
    per_device_train_batch_size = 4,
    # per_device_eval_batch_size=2,
    max_steps = 100
    )

trainer = SFTTrainer(
    model=model,
    args = training_arguments,
    tokenizer = tokenizer,
    train_dataset = load_dataset(path="vivekdugale/falcon7B_filtered_dataset_amod_helios_450", split='train'),
    peft_config = LoraConfig(task_type="CAUSAL_LM", r=64, lora_alpha=16, lora_dropout=0.1),
    dataset_text_field = "text"
    )

trainer.train()

text_gen_pipeline = pipeline(task = "text-generation", model = model, tokenizer = tokenizer, max_length = 100)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [3]:

!pip install pyngrok

In [ ]:
# port_no=4000
# import requests
# from flask import Flask
# import requests
# from pyngrok import ngrok

# app = Flask(__name__)
# ngrok.set_auth_token("2eBZAe1pQCGMPK2iqfieQR0jLaO_2ssxCssXZDckNQFWU6F9b")
# public_url=ngrok.connect(port_no)

# print("NGROK - ", public_url)

In [ ]:
import psutil

def terminate_processes_on_port(port):
    for proc in psutil.process_iter(['pid', 'name', 'connections']):
        for conn in proc.info.get('connections', []):
            if conn.laddr and conn.laddr.port == port:
                print(f"Terminating process {proc.pid} - {proc.name()}")
                proc.terminate()

if __name__ == "__main__":
    port_to_terminate = 6000
    terminate_processes_on_port(port_to_terminate)


In [ ]:
from flask import Flask, jsonify, request
from pyngrok import ngrok
import threading

port = 6000

app = Flask(__name__)
ngrok.set_auth_token("2cTqZ40Esa1XYycw883DFQ3FkXS_4TMM2qM28xDcJekSLTZz6")
public_url = ngrok.connect(port).public_url
print("NGROK ", public_url)
app.secret_key='password'

@app.route('/', methods=['POST'])
def llm():
    prompt = request.json['prompt']
    ans = text_gen_pipeline(f"[INST] {prompt} [/INST]")
    return jsonify({'text': ans[0]['generated_text']})

threading.Thread(target=app.run(port = port), kwargs={"use_reloader": False}).start()


NGROK  https://9146-34-133-176-153.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:6000
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [09/May/2024 14:39:10] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 14:40:12] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 14:40:54] "POST / HTTP/1.1" 200 -


In [ ]:
from flask import Flask, jsonify, request
from pyngrok import ngrok
import requests

ngrok.set_auth_token("2eBoiQtNCe4zdUtOmBnl4VLUiJz_7UADmPhccHV2WQwrXyNgw")
public_url = ngrok.connect(4000).public_url
print("NGROK - ", public_url)

In [ ]:
import requests
from flask import Flask

app=Flask(__name__)
# Assuming the Flask app is running on localhost and port 5000
url = "http://localhost:5000/chatbot"

@app.route("/")
def route():
# Send a GET request to the Flask app
    response = requests.get(url)
    print("HELLO")
    # return {"In":"Local host 5000"}

# Check if the request was successful (status code 200)
    if response.status_code == 200:
    # Extract the JSON data from the response
        data = response.json()
        print(data)
        return {"index":"Successful"}
    else:
        print("SOMETHING ELSE")
        return{"Error": response.status_code}

app.run(debug=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save("/content/drive/MyDrive/llm.h5")